Testing connection and API keys

In [12]:
from dotenv import load_dotenv
load_dotenv()

import os

from openai import AsyncOpenAI
from anthropic import AsyncAnthropic
from exa_py import AsyncExa

Test OpenAI

In [ ]:
client = AsyncOpenAI()

response = await client.responses.create(
  model="gpt-4.1",
  input="Tell me a three sentence bedtime story about a unicorn."
)

print(response.output_text)

Under the silvery light of the moon, a gentle unicorn named Luna danced through a meadow sparkling with magical dew. She used her shimmering horn to paint rainbows across the night sky, filling the world with color and sweet dreams. As the stars twinkled above, Luna curled up in a soft patch of clover, knowing that tomorrow would bring even more adventures.


Test Anthropic

In [10]:
client = AsyncAnthropic()

message = await client.messages.create(
    max_tokens=1024,
    messages=[{
        "content": "Tell me a three sentence bedtime story about a unicorn.",
        "role": "user",
    }],
    model="claude-sonnet-4-5-20250929",
)

print(message.content[0].text)

Luna the unicorn galloped through moonlit meadows, her silver horn glowing softly in the night. She sprinkled stardust over sleeping flowers, helping them dream of spring. As dawn approached, she nestled beside a babbling brook and drifted into peaceful sleep, wrapped in morning mist.


Test EXA

In [17]:
exa = AsyncExa(api_key=os.environ.get("EXA_API_KEY"))

result = await exa.search(
  "hottest AI startups",
  num_results=2
)

In [39]:
result.results[0].title

'Hottest AI Startups in the USA - LinkedIn'

In [40]:
result.results[0].url

'https://www.linkedin.com/pulse/hottest-ai-startups-usa-aitechsupports-ouwfc'

In [37]:
result.results[0].published_date

'2025-12-04T12:04:50.000Z'

In [43]:
print(result.results[0].text[:500])

Hottest AI Startups in the USA
Agree & Join LinkedIn
By clicking Continue to join or sign in, you agree to LinkedIn’s[User Agreement](https://www.linkedin.com/legal/user-agreement?trk=linkedin-tc_auth-button_user-agreement),[Privacy Policy](https://www.linkedin.com/legal/privacy-policy?trk=linkedin-tc_auth-button_privacy-policy), and[Cookie Policy](https://www.linkedin.com/legal/cookie-policy?trk=linkedin-tc_auth-button_cookie-policy).
``````````````
![]()## Sign in to view more content
Create y


In [48]:
answer_with_text = await exa.answer(
  query = "What are the hottest AI startups?",
  model = "exa-pro",
  text = True
)

In [51]:
answer_with_text.answer

'As of December 22, 2025, some of the hottest AI startups include Thinking Machines Lab, valued at $12 billion, and Glean, valued at $7.2 billion ([Reuters](https://www.reuters.com/technology/mira-muratis-ai-startup-thinking-machines-raises-2-billion-a16z-led-round-2025-07-15), [Reuters](https://www.reuters.com/technology/ai-company-glean-hits-72-billion-valuation-latest-funding-round-2025-06-10)). Other notable companies mentioned as "hottest" or "most promising" in 2025 include AI Squared, Morphos AI, and Writer ([CRN](https://www.crn.com/news/ai/2025/the-10-hottest-ai-startup-companies-of-2025-so-far)).'

EXA WebSets

https://docs.exa.ai/websets/api/get-started

In [60]:
from exa_py.websets.types import CreateWebsetParameters, CreateEnrichmentParameters
from exa_py import Exa

exa = Exa(os.getenv('EXA_API_KEY'))

In [77]:
preview = exa.websets.preview(params={
    'query': 'Latest AI research papers that are applicable to private equity'
})

In [78]:
preview.search.entity

WebsetResearchPaperEntity(type='research_paper')

In [79]:
for e, i in enumerate(preview.search.criteria):
    print(f"{e+1}: {i.description}")
    print("===" * 30)

1: Research paper focused on artificial intelligence (AI) applications or relevance to private equity
2: Published during 2024 or 2025


arXiv

In [80]:
import urllib.request
import urllib.parse
import feedparser
from datetime import datetime, timedelta

In [81]:
def search_arxiv(
    search_query: str = "",
    categories: list[str] | None = None,
    start: int = 0,
    max_results: int = 10,
    sort_by: str = "submittedDate",  # relevance, lastUpdatedDate, submittedDate
    sort_order: str = "descending"   # ascending, descending
) -> dict:
    """
    Search arXiv API with optional category filters.
    
    Categories examples:
        - cs.AI (Artificial Intelligence)
        - cs.CL (Computation and Language)
        - cs.LG (Machine Learning)
        - cs.CV (Computer Vision)
        - stat.ML (Machine Learning - Statistics)
        - q-fin.GN (General Finance)
    """
    base_url = "http://export.arxiv.org/api/query?"
    
    # Build the search query
    query_parts = []
    
    # Add free-text search if provided
    if search_query:
        query_parts.append(f"all:{search_query}")
    
    # Add category filter (OR between categories)
    if categories:
        cat_query = " OR ".join([f"cat:{cat}" for cat in categories])
        if len(categories) > 1:
            cat_query = f"({cat_query})"
        query_parts.append(cat_query)
    
    # Combine with AND
    full_query = " AND ".join(query_parts) if query_parts else "all:*"
    
    params = {
        "search_query": full_query,
        "start": start,
        "max_results": max_results,
        "sortBy": sort_by,
        "sortOrder": sort_order
    }
    
    url = base_url + urllib.parse.urlencode(params)
    print(f"Query URL: {url}\n")
    
    # Fetch and parse
    response = urllib.request.urlopen(url)
    feed = feedparser.parse(response.read().decode("utf-8"))
    
    return feed


def print_results(feed: dict):
    """Pretty print arXiv search results."""
    print(f"Total results: {feed.feed.get('opensearch_totalresults', 'N/A')}")
    print(f"Showing: {len(feed.entries)} entries\n")
    print("=" * 80)
    
    for i, entry in enumerate(feed.entries, 1):
        print(f"\n[{i}] {entry.title}")
        print(f"    ID: {entry.id}")
        print(f"    Published: {entry.published}")
        print(f"    Authors: {', '.join(a.name for a in entry.authors[:3])}", end="")
        if len(entry.authors) > 3:
            print(f" + {len(entry.authors) - 3} more")
        else:
            print()
        
        # Categories
        categories = [tag.term for tag in entry.tags]
        print(f"    Categories: {', '.join(categories)}")
        
        # Truncated summary
        summary = entry.summary.replace("\n", " ")[:200]
        print(f"    Summary: {summary}...")
        print("-" * 80)

In [ ]:
# =============================================================================
# Example: Search across multiple categories (AI, NLP, Finance)
# =============================================================================
print("\n\n🔍 Example: AI + NLP + Finance papers\n")
feed = search_arxiv(
    categories=["cs.AI", "cs.CL", "q-fin.GN"],
    max_results=5,
    sort_by="submittedDate",
    sort_order="descending"
)
print_results(feed)



🔍 Example 2: AI + NLP + Finance papers

Query URL: http://export.arxiv.org/api/query?search_query=%28cat%3Acs.AI+OR+cat%3Acs.CL+OR+cat%3Aq-fin.GN%29&start=0&max_results=5&sortBy=submittedDate&sortOrder=descending

Total results: 223367
Showing: 5 entries


[1] Re-Depth Anything: Test-Time Depth Refinement via Self-Supervised Re-lighting
    ID: http://arxiv.org/abs/2512.17908v1
    Published: 2025-12-19T18:59:56Z
    Authors: Ananta R. Bhattarai, Helge Rhodin
    Categories: cs.CV, cs.AI, cs.LG
    Summary: Monocular depth estimation remains challenging as recent foundation models, such as Depth Anything V2 (DA-V2), struggle with real-world images that are far from the training distribution. We introduce...
--------------------------------------------------------------------------------

[2] Adversarial Robustness of Vision in Open Foundation Models
    ID: http://arxiv.org/abs/2512.17902v1
    Published: 2025-12-19T18:59:16Z
    Authors: Jonathon Fox, William J Buchanan, Pavlos Papa

In [84]:
# =============================================================================
# Example 3: Keyword search within specific categories
# =============================================================================
print("\n\n🔍 Example 3: 'LLM' or 'transformer' in AI/ML categories\n")
feed = search_arxiv(
    search_query="LLM OR transformer OR \"large language model\"",
    categories=["cs.AI", "cs.CL", "cs.LG"],
    max_results=5,
    sort_by="submittedDate",
    sort_order="descending"
)
print_results(feed)



🔍 Example 3: 'LLM' or 'transformer' in AI/ML categories

Query URL: http://export.arxiv.org/api/query?search_query=all%3ALLM+OR+transformer+OR+%22large+language+model%22+AND+%28cat%3Acs.AI+OR+cat%3Acs.CL+OR+cat%3Acs.LG%29&start=0&max_results=5&sortBy=submittedDate&sortOrder=descending

Total results: 210238
Showing: 5 entries


[1] Diffusion Forcing for Multi-Agent Interaction Sequence Modeling
    ID: http://arxiv.org/abs/2512.17900v1
    Published: 2025-12-19T18:59:02Z
    Authors: Vongani H. Maluleke, Kie Horiuchi, Lea Wilken + 3 more
    Categories: cs.CV, cs.RO
    Summary: Understanding and generating multi-person interactions is a fundamental challenge with broad implications for robotics and social computing. While humans naturally coordinate in groups, modeling such ...
--------------------------------------------------------------------------------

[2] XAgen: An Explainability Tool for Identifying and Correcting Failures in Multi-Agent Workflows
    ID: http://arxiv.org/ab

In [ ]:
print("\n\n🔍 Example 3: 'LLM' or 'transformer' in AI/ML categories\n")
feed = search_arxiv(
    search_query="LLM OR transformer OR \"large language model\"",
    categories=["cs.AI", "cs.CL", "cs.LG"],
    max_results=5,
    sort_by="submittedDate",
    sort_order="descending"
)
print_results(feed)